In [58]:
#Requierd Libraries
from pathlib import Path
from ultralytics import YOLO

In [59]:
# Ultralytics YOLO 🚀, AGPL-3.0 license

import torch  
from pathlib import Path
from PIL import Image  # For loading images

from ultralytics import YOLO


def auto_annotate_for_detection(data, det_model="yolov8x.pt", device="", output_dir=None):
  """
  Automatically annotates images using a YOLO object detection model.

  Args:
    data (str): Path to a folder containing images to be annotated.
    det_model (str, optional): Pre-trained YOLO detection model. Defaults to 'yolov8x.pt'.
    device (str, optional): Device to run the model on. 'mps' for Apple Silicon, 'cuda' for GPU, or '' for CPU.
    output_dir (str | None | optional): Directory to save the annotated results.
      Defaults to a 'labels' folder in the same directory as 'data'.
  """

  # Check if MPS is available (for Apple Silicon)
  if device == 'mps' and not torch.backends.mps.is_available():
    print("MPS device requested but not available. Defaulting to CPU.")
    device = '' 

  det_model = YOLO(det_model).to(device) 

  data = Path(data)
  if not output_dir:
    output_dir = data.parent / f"{data.stem}_labels" 
  Path(output_dir).mkdir(exist_ok=True, parents=True)

  det_results = det_model(data, stream=True)  

  for result in det_results:
    boxes = result.boxes  
    confidences = boxes.conf.tolist() 
    class_ids = boxes.cls.int().tolist() 

    with open(f"{Path(output_dir) / Path(result.path).stem}.txt", "w") as f:
      for i in range(len(boxes)):
        x_center, y_center, width, height = boxes[i].xyxy[0].tolist()

        # ***Solution 1: Loading Image from 'result.path'***
        if hasattr(result, 'path'): 
          img = Image.open(result.path)
          img_height, img_width = img.size

        # ***Solution 2: Accessing Image from 'result' Object (Modify if needed)***
        elif hasattr(result, 'img'):  # Replace 'img' with the correct attribute 
          img_height, img_width = result.img.shape[:2] 
        else:
          raise ValueError("Cannot find image data in 'result' object. Please check its structure.")

        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        f.write(f"{class_ids[i]} {x_center} {y_center} {width} {height} \n")

In [60]:
images_path = "./images/train"
model_path = "./2dModel.pt"
output_path = "./labels/train"

In [61]:
auto_annotate_for_detection(data=images_path, det_model=model_path, device='mps', output_dir=output_path)


image 1/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/0.jpg: 384x640 1 car, 2 SUVs, 34.6ms
image 2/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/1.jpg: 384x640 1 car, 2 SUVs, 22.6ms
image 3/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/10.jpg: 384x640 1 car, 2 SUVs, 21.1ms
image 4/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/100.jpg: 384x640 2 SUVs, 1 motorcycle, 18.8ms
image 5/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/1000.jpg: 384x640 1 car, 2 SUVs, 17.1ms
image 6/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/1001.jpg: 384x640 1 car, 2 SUVs, 19.2ms
image 7/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/1002.jpg: 384x640 1 car, 2 SUVs, 20.0ms
image 8/2064 /Users/Robin1/Desktop/LIDAR Project/Zelzah and Plummer Dataset/images/train/1003.jpg: